In [60]:
import json
import pandas as pd
import numpy as np
import re

In [20]:
with open('../data/housing_data_2020_03_28.json', 'r') as s:
    housesdict = json.load(s)

In [71]:
def strip_int_column(dataf, columns):
    for c in columns:
        cvalues = list(dataf[c].values)
        cvalues = [re.sub('[^0-9]','', x) for x in cvalues]
        dataf[f'{c}_clean'] = [0 if y == '' else int(y) for y in cvalues]
    return dataf

def prepare_housing_dict(dicty):
    
    # merge dict into pandas frame
    housesdf = pd.DataFrame.from_dict(dicty, orient='index')
    
    # split location into zip and citu
    housesdf['zipcode'] = housesdf['location'].str.split(',').str[0]
    housesdf['city'] = housesdf['location'].str.split(',').str[1]
    housesdf['city'] = housesdf['city'].fillna('noaddress')
    
    # convert given columns to ints
    housesdf = strip_int_column(housesdf, ['price', 'zipcode', 'meters'])
    
    # divide extreem high prices by 1000 to correct for dtype error in source
    housesdf.loc[lambda x: x['price_clean']>99999999, 'price_clean'] = housesdf['price_clean']/1000
    
    # calculate meter price on row level and replace failed calculation with 0
    housesdf['meter_price'] = housesdf['price_clean'] / housesdf['meters_clean']
    housesdf['meter_price'] = housesdf['meter_price'].replace([np.inf, -np.inf], 0)
    
    # filter on available houses only & houses with at least the field city filled
    housesdf = housesdf.loc[lambda x: x['status']=='available']
    housesdf = housesdf.loc[lambda x: x['location']!='noaddress']
    
    return housesdf

In [72]:
housesdf = prepare_housing_dict(housesdict)

In [84]:
len(housesdf)

51641

In [80]:
housesdf.head()

,location,price,meters,sold,zipcode,city,price_clean,zipcode_clean,meters_clean,meter_price
0,"1083 KA, Amsterdam",€ 925.000 k.k.,211 m²,available,1083 KA,Amsterdam,925000.0,1083,211,4383.886256
1,"1741 DJ, Schagen",€ 285.000 k.k.,211 m²,available,1741 DJ,Schagen,285000.0,1741,211,1350.710900
2,"1567 HD, Assendelft",€ 346.000 k.k.,211 m²,available,1567 HD,Assendelft,346000.0,1567,211,1639.810427
4,"3267 AM, Goudswaard",€ 390.000 k.k.,211 m²,available,3267 AM,Goudswaard,390000.0,3267,211,1848.341232
5,"1508 WE, Zaandam",€ 215.000 k.k.,211 m²,available,1508 WE,Zaandam,215000.0,1508,211,1018.957346


In [89]:
average_price = round(housesdf.loc[lambda x: (x['price_clean']>0)]['price_clean'].mean(),2 )
median_price = round(housesdf.loc[lambda x: (x['price_clean']>0)]['price_clean'].median(), 2)
average_meters = round(housesdf.loc[lambda x: (x['meters_clean']>0)]['meters_clean'].mean(), 2)
median_meters = round(housesdf.loc[lambda x: (x['meters_clean']>0)]['meters_clean'].median(), 2)
average_meter_price = round(housesdf.loc[lambda x: (x['meter_price']>0)]['meter_price'].mean(), 2)
median_meter_price = round(housesdf.loc[lambda x: (x['meter_price']>0)]['meter_price'].median(), 2)
n_houses = housesdf['city'].count()
print('Netherlands area:')
print(f'====> Houses available: {n_houses}')
print(f'====> Average price: {average_price}')
print(f'====> Median price: {median_price}')
print(f'====> Average meters: {average_meters}')
print(f'====> Median meters: {median_meters}')
print(f'====> Average meter price: {average_meter_price}')
print(f'====> Median meter price: {median_meter_price}')

def city_sum(housesdf, cities):
    for city in cities:
        city_housesdf = housesdf.loc[lambda x: x['city'].str.contains(city)]
        average_price = round(city_housesdf.loc[lambda x: (x['price_clean']>0)]['price_clean'].mean(), 2)
        median_price = round(city_housesdf.loc[lambda x: (x['price_clean']>0)]['price_clean'].median(), 2)
        average_meters = round(city_housesdf.loc[lambda x: (x['meters_clean']>0)]['meters_clean'].mean(), 2)
        median_meters = round(city_housesdf.loc[lambda x: (x['meters_clean']>0)]['meters_clean'].median(), 2)
        average_meter_price = round(city_housesdf.loc[lambda x: (x['meter_price']>0)]['meter_price'].mean(), 2)
        median_meter_price = round(city_housesdf.loc[lambda x: (x['meter_price']>0)]['meter_price'].median(), 2)
        n_houses = city_housesdf.loc[lambda x: x['city'].str.contains(city)]['city'].count()
        print(f'{city} area:')
        print(f'====> Houses available: {n_houses}')
        print(f'====> Average price: {average_price}')
        print(f'====> Median price: {median_price}')
        print(f'====> Average meters: {average_meters}')
        print(f'====> Median meters: {median_meters}')
        print(f'====> Average meter price: {average_meter_price}')
        print(f'====> Median meter price: {median_meter_price}')
        
cities = ['Amsterdam', 'Rotterdam', 'Utrecht', 'Hilversum', 'Amstelveen']
city_sum(housesdf, cities)

Netherlands area:
====> Houses available: 51641
====> Average price: 448247.66
====> Median price: 349000.0
====> Average meters: 143.42
====> Median meters: 125.0
====> Average meter price: 3684.8
====> Median meter price: 2733.58
Amsterdam area:
====> Houses available: 1661
====> Average price: 733219.22
====> Median price: 525000.0
====> Average meters: 140.34
====> Median meters: 122.0
====> Average meter price: 6059.79
====> Median meter price: 4306.57
Rotterdam area:
====> Houses available: 1056
====> Average price: 464835.87
====> Median price: 324750.0
====> Average meters: 135.92
====> Median meters: 118.0
====> Average meter price: 3845.83
====> Median meter price: 2840.91
Utrecht area:
====> Houses available: 489
====> Average price: 483461.98
====> Median price: 400000.0
====> Average meters: 135.1
====> Median meters: 124.0
====> Average meter price: 4118.69
====> Median meter price: 3339.2
Hilversum area:
====> Houses available: 306
====> Average price: 693917.06
====> Me